## Run this tutorial with the **nnpcg_ana** environment

### Importing libraries

In [2]:
import os
import matplotlib
import numpy as np
from htmd.ui import *
import seaborn as sns
import matplotlib.pyplot as plt
from nnpcg.analyzer import EasyAnalyzer
from nnpcg.analyzer import sample_state, build_backbone, plotContour

### Step 1: Loading the dataset

In [ ]:
# modify the next variable with the path to the folder where you cloned the repository
path_to_repo = '/home/dpastor/Dropbox (Biocomputing)/GitHub/projects/nnpcg'

# defining the path to the analysis folder
analysis_data_folder = os.path.join(path_to_repo, 'nnpcg/data/analysis')

# creating the EasyAnalyzer object
analysis_test = EasyAnalyzer(refmodel=os.path.join(analysis_data_folder, 'proteing_1mi0.pdb'), reftic=os.path.join(analysis_data_folder, 'proteing_TICA_lag20_CAdist_skip1.dat'), refmodel=os.path.join(analysis_data_folder, 'proteing_model_lag10ns_1200k_4ticadim_CAdist_skip1.dat'))

### Step 2: Plotting the TICA projection for the all-atom reference

In [1]:
# The figure depicted illustrates the estimation of the free-energy surface obtained through MD simulations as a 
# reference. We will employ this as a benchmark to assess and contrast the results of our coarse-grained simulations
# executed using the trained potential.
analysis_test.plot_ref_tic()

### Step 3: Loading the resulting CG trajectories

In [ ]:
sims = simlist(glob(analysis_data_folder + '/proteing_32trajs_250_ts1/*'), os.path.join(analysis_data_folder, 'proteing_ca_top_dih.pdb'))
analysis_test.load_cg_data(sims)

### Step 4: Dimensionality reduction of the CG simulations

In [ ]:
analysis_test.dimensionality_reduction()

### Step 5: Data clustering and Markov state model estimation

In [ ]:
# Clustering the data
analysis_test.data_clustering()

# Creating the Markov model
analysis_test.cg_markov_model()

### Step 6: Comparing potential energy surfaces
After constructing the Markov state model, we can compare the estimated free energy surface to the one obtained from the reference molecular dynamics (MD) simulations. In this presentation, we include plots for both 2D and 1D potential energy surfaces. To estimate the free energy surface, we first histogram the projected TICA space into 80 bins, and then we reweight the histogram using the stationary distribution of the Markov state model.

#### A: Plotting 2D projections of the potential energy surfaces

In [ ]:
# plotting parameters
levels = np.arange(0, 7.6, 1.5)
cmap = 'viridis'
dimx, dimy = 0,1
states = list(range(analysis_test.cgmodel.macronum))[::-1]
refstates = list(range(analysis_test.refmodel.macronum))[::-1]

# Creating the figure
fig, ax = plt.subplots(ncols=2, figsize=[26,11])

# Coarse-grained simulations
plt.sca(ax[0])
plotContour(np.concatenate(analysis_test.cgmodel.data.dat), analysis_test.cgweights, levels, dimx=dimx, dimy=dimy, cmap=cmap)
cbar = plt.colorbar()
cbar.ax.tick_params() 
cbar.ax.get_yaxis().labelpad = 20
cbar.ax.set_ylabel('kcal/mol', rotation=270)
plt.xlabel(f'TICA dim {dimx}', size=16)
plt.ylabel(f'TICA dim {dimy}', size=16)
plt.title('CG simulations- {:.1f}µs'.format(np.concatenate(analysis_test.cgmodel.data.dat).shape[0] * 0.002))

# Reference MD simulations
plt.sca(ax[1])
plotContour(np.concatenate(analysis_test.refmodel.data.dat), analysis_test.mweights, levels, dimx=dimx, dimy=dimy, cmap=cmap)
cbar = plt.colorbar()
cbar.ax.tick_params() 
cbar.ax.get_yaxis().labelpad = 20
cbar.ax.set_ylabel('kcal/mol', rotation=270)
plt.xlabel(f'TICA dim {dimx}', size=16)
plt.ylabel(f'TICA dim {dimy}', size=16)
plt.title('Reference MD simulations - {:.1f}µs'.format(np.concatenate(analysis_test.refmodel.data.dat).shape[0] * 0.1))
plt.show()

#### B: Plotting 1D projections of the potential energy surfaces

In [ ]:
# 1D TICA
def get1dtica(data, weights, bins=120, dim=0):
    counts, bins = np.histogram(data[:,dim], bins=bins, weights=weights)
    energy =  -Kinetics._kB*300*np.log(counts)
    ecorr = np.min(energy[energy!=0])
    energy = energy - ecorr
    return energy, bins

bins=120
ref_energy, ref_bins = get1dtica(np.concatenate(analysis_test.refmodel.data.dat), analysis_test.mweights, bins=bins)
cg_energy, cg_bins = get1dtica(np.concatenate(analysis_test.cgmodel.data.dat), analysis_test.cgweights, bins=bins)
crystal_dist = MetricSelfDistance('name CA', pbc=None).project(analysis_test.refmol)
analysis_test.reftic.set_params(dim=3)
tica_crystal = analysis_test.reftic.transform(crystal_dist).flatten()

# comparing the 1D TICAs
plt.figure(figsize=[8,8])
sns.lineplot(x=ref_bins[:-1], y=ref_energy, label='Ref', alpha=0.5)
plt.fill_between(ref_bins[:-1], ref_energy, alpha=0.3)
sns.lineplot(x=cg_bins[:-1], y=cg_energy, label='CG', color='lightgreen')
plt.fill_between(cg_bins[:-1], cg_energy, alpha=0.3, color='lightgreen')
plt.scatter(tica_crystal[0], -0.15, s=40, marker='|', c='red', label='Crystal')

plt.xlabel(f'TIC 1')
plt.ylabel(f'Free energy (kcal/mol)')
plt.title(f'1D TICA')
plt.legend(loc='upper center')
plt.show()